# Evolver Loop 10 - LB Feedback Analysis

**LB Score: 70.6305** (matches CV exactly - perfect calibration!)

The saspav_best.csv source improved our score from 70.6473 to 70.6305 (0.017 improvement).

Now we need to expand the ensemble with MORE sources from the jonathanchan kernel list.

In [1]:
import numpy as np
import pandas as pd
import glob
import os
from shapely import affinity
from shapely.geometry import Polygon
from decimal import Decimal, getcontext

getcontext().prec = 25
scale_factor = Decimal('1e15')

In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

In [3]:
def calculate_score(trees):
    """Calculate score for a list of trees"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def has_collision(trees):
    """Check for collisions between trees using Shapely"""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_trees_from_df(df, n):
    """Load trees for a specific N from dataframe"""
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:] if str(row['x']).startswith('s') else str(row['x'])
        y = str(row['y'])[1:] if str(row['y']).startswith('s') else str(row['y'])
        deg = str(row['deg'])[1:] if str(row['deg']).startswith('s') else str(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees, group_data

In [4]:
# Collect ALL available CSV files from all sources
all_csv_paths = []

# Original sources
original_sources = [
    '/home/code/exploration/datasets/submission.csv',
    '/home/code/exploration/datasets/santa-2025.csv',
    '/home/code/exploration/datasets/saspav_best.csv',
    '/home/code/exploration/datasets/smartmanoj.csv',
    '/home/code/exploration/datasets/submission_best.csv',
]
all_csv_paths.extend(original_sources)

# Telegram sources
telegram_csvs = glob.glob('/home/code/exploration/datasets/telegram/*.csv')
all_csv_paths.extend(telegram_csvs)

# Santa25-public sources
santa25_public_csvs = glob.glob('/home/code/exploration/datasets/santa25_public/*.csv')
all_csv_paths.extend(santa25_public_csvs)

# Seowoohyeon sources
seowoohyeon_csvs = glob.glob('/home/code/exploration/datasets/seowoohyeon/*.csv')
all_csv_paths.extend(seowoohyeon_csvs)

print(f'Total CSV files found: {len(all_csv_paths)}')
for p in all_csv_paths:
    print(f'  {p}')

Total CSV files found: 25
  /home/code/exploration/datasets/submission.csv
  /home/code/exploration/datasets/santa-2025.csv
  /home/code/exploration/datasets/saspav_best.csv
  /home/code/exploration/datasets/smartmanoj.csv
  /home/code/exploration/datasets/submission_best.csv
  /home/code/exploration/datasets/telegram/72.49.csv
  /home/code/exploration/datasets/telegram/71.97.csv
  /home/code/exploration/datasets/santa25_public/submission_JKoT4.csv
  /home/code/exploration/datasets/santa25_public/New_Tree_144_196.csv
  /home/code/exploration/datasets/santa25_public/submission_JKoT3.csv
  /home/code/exploration/datasets/santa25_public/santa2025_ver2_v61.csv
  /home/code/exploration/datasets/santa25_public/submission_JKoT2.csv
  /home/code/exploration/datasets/santa25_public/santa2025_ver2_v67.csv
  /home/code/exploration/datasets/santa25_public/santa2025_ver2_v76.csv
  /home/code/exploration/datasets/santa25_public/submission_70_936673758122.csv
  /home/code/exploration/datasets/santa25

In [5]:
# Load all sources and calculate scores for each N
print('Loading all sources...')
sources = {}

for path in all_csv_paths:
    try:
        df = pd.read_csv(path)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            print(f'  Skipping {path} - missing columns')
            continue
        name = os.path.basename(path)
        sources[name] = df
        print(f'  Loaded {name}: {len(df)} rows')
    except Exception as e:
        print(f'  Error loading {path}: {e}')

print(f'\nTotal sources loaded: {len(sources)}')

Loading all sources...
  Loaded submission.csv: 20100 rows
  Loaded santa-2025.csv: 20100 rows
  Loaded saspav_best.csv: 20100 rows
  Loaded smartmanoj.csv: 20100 rows
  Loaded submission_best.csv: 20100 rows
  Loaded 72.49.csv: 20100 rows
  Loaded 71.97.csv: 20100 rows


  Loaded submission_JKoT4.csv: 20100 rows
  Loaded New_Tree_144_196.csv: 20100 rows
  Loaded submission_JKoT3.csv: 20100 rows
  Loaded santa2025_ver2_v61.csv: 20100 rows
  Loaded submission_JKoT2.csv: 20100 rows
  Loaded santa2025_ver2_v67.csv: 20100 rows
  Loaded santa2025_ver2_v76.csv: 20100 rows
  Loaded submission_70_936673758122.csv: 20100 rows
  Loaded santa2025_ver2_v65.csv: 20100 rows


  Loaded submission_70_926149550346.csv: 20100 rows
  Loaded santa2025_ver2_v66.csv: 20100 rows
  Loaded santa2025_ver2_v63.csv: 20100 rows
  Loaded santa2025_ver2_v69.csv: 20100 rows
  Loaded submission_JKoT1.csv: 20100 rows
  Loaded submission_opt1.csv: 20100 rows
  Loaded santa2025_ver2_v68.csv: 20100 rows
  Loaded submission.csv: 20100 rows
  Loaded submission_sa.csv: 20100 rows

Total sources loaded: 24


In [6]:
# Calculate scores for each N from each source
print('Calculating scores for each N from each source...')
scores_by_source = {name: {} for name in sources}

for name, df in sources.items():
    for n in range(1, 201):
        try:
            trees, _ = load_trees_from_df(df, n)
            if len(trees) == n:
                score = calculate_score(trees)
                scores_by_source[name][n] = score
        except Exception as e:
            pass
    print(f'  {name}: {len(scores_by_source[name])} N values')

print('Done calculating scores.')

Calculating scores for each N from each source...


  submission.csv: 200 N values


  santa-2025.csv: 200 N values


  saspav_best.csv: 200 N values


  smartmanoj.csv: 200 N values


...

  santa2025_ver2_v69.csv: 200 N values


  submission_JKoT1.csv: 200 N values


  submission_opt1.csv: 200 N values


  santa2025_ver2_v68.csv: 200 N values


  submission_sa.csv: 200 N values
Done calculating scores.


In [7]:
# Find the best source for each N (by score, without overlap check yet)
print('Finding best source for each N by score...')
best_by_n = {}

for n in range(1, 201):
    best_score = float('inf')
    best_source = None
    
    for name in sources:
        if n in scores_by_source[name]:
            score = scores_by_source[name][n]
            if score < best_score:
                best_score = score
                best_source = name
    
    if best_source:
        best_by_n[n] = {'source': best_source, 'score': best_score}

# Count wins by source
wins_by_source = {}
for n, info in best_by_n.items():
    src = info['source']
    wins_by_source[src] = wins_by_source.get(src, 0) + 1

print('\nWins by source (before overlap validation):')
for src, wins in sorted(wins_by_source.items(), key=lambda x: -x[1]):
    print(f'  {src}: {wins} N values')

Finding best source for each N by score...

Wins by source (before overlap validation):
  saspav_best.csv: 122 N values
  santa-2025.csv: 77 N values
  submission.csv: 1 N values


In [8]:
# Now validate with overlap check and create ensemble
print('\nValidating with overlap check...')
baseline_df = pd.read_csv('/home/code/exploration/datasets/submission.csv')

ensemble_rows = []
ensemble_sources = {}
ensemble_scores = {}
validation_failures = 0

for n in range(1, 201):
    best_valid_score = float('inf')
    best_valid_source = None
    best_valid_data = None
    
    # Try each source in order of score
    candidates = []
    for name in sources:
        if n in scores_by_source[name]:
            candidates.append((scores_by_source[name][n], name))
    candidates.sort()  # Sort by score
    
    for score, name in candidates:
        try:
            trees, data = load_trees_from_df(sources[name], n)
            if len(trees) == n and not has_collision(trees):
                best_valid_score = score
                best_valid_source = name
                best_valid_data = data
                break
        except Exception as e:
            continue
    
    # Fallback to baseline if no valid source found
    if best_valid_data is None:
        trees, data = load_trees_from_df(baseline_df, n)
        best_valid_score = calculate_score(trees)
        best_valid_source = 'submission.csv (fallback)'
        best_valid_data = data
        validation_failures += 1
    
    ensemble_rows.append(best_valid_data)
    ensemble_sources[n] = best_valid_source
    ensemble_scores[n] = best_valid_score

print(f'Validation failures (fell back to baseline): {validation_failures}')

# Count wins by source after validation
wins_after_validation = {}
for n, src in ensemble_sources.items():
    wins_after_validation[src] = wins_after_validation.get(src, 0) + 1

print('\nWins by source (after overlap validation):')
for src, wins in sorted(wins_after_validation.items(), key=lambda x: -x[1]):
    print(f'  {src}: {wins} N values')


Validating with overlap check...


Validation failures (fell back to baseline): 0

Wins by source (after overlap validation):
  saspav_best.csv: 122 N values
  santa-2025.csv: 68 N values
  smartmanoj.csv: 9 N values
  71.97.csv: 1 N values


In [9]:
# Calculate total score
total_score = sum(ensemble_scores.values())
print(f'\nTotal ensemble score: {total_score:.6f}')
print(f'Previous best (saspav ensemble): 70.630478')
print(f'Improvement: {70.630478 - total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap to target: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)')


Total ensemble score: 70.630603
Previous best (saspav ensemble): 70.630478
Improvement: -0.000125
Target: 68.919154
Gap to target: 1.711449 (2.48%)


In [10]:
# Create submission file
ensemble_df = pd.concat(ensemble_rows, ignore_index=True)

# Sort by N and index
ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]

# Save
output_path = '/home/code/submission_candidates/candidate_010.csv'
ensemble_df.to_csv(output_path, index=False)
print(f'Saved ensemble to {output_path}')
print(f'Total rows: {len(ensemble_df)}')

Saved ensemble to /home/code/submission_candidates/candidate_010.csv
Total rows: 20100


In [11]:
# Verify the saved file
verify_df = pd.read_csv(output_path)
verify_total = 0
for n in range(1, 201):
    trees, _ = load_trees_from_df(verify_df, n)
    score = calculate_score(trees)
    verify_total += score

print(f'Verified total score: {verify_total:.6f}')
print(f'Matches calculated: {abs(verify_total - total_score) < 0.0001}')

Verified total score: 70.630603
Matches calculated: True


In [12]:
# Show which N values improved vs previous best
print('\nN values that improved vs saspav ensemble:')
prev_best_df = pd.read_csv('/home/code/submission_candidates/candidate_009.csv')

improvements = []
for n in range(1, 201):
    prev_trees, _ = load_trees_from_df(prev_best_df, n)
    prev_score = calculate_score(prev_trees)
    new_score = ensemble_scores[n]
    
    if new_score < prev_score - 1e-9:
        improvements.append((n, prev_score, new_score, prev_score - new_score, ensemble_sources[n]))

print(f'Total improvements: {len(improvements)}')
for n, prev, new, diff, src in sorted(improvements, key=lambda x: -x[3])[:20]:
    print(f'  N={n}: {prev:.6f} -> {new:.6f} (improvement: {diff:.6f}) from {src}')


N values that improved vs saspav ensemble:


Total improvements: 0


In [13]:
# Save metrics
import json

metrics = {
    'cv_score': total_score,
    'sources_used': dict(wins_after_validation),
    'improvements_over_previous': len(improvements),
    'validation_failures': validation_failures
}

with open('/home/code/experiments/011_expanded_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print('Metrics saved.')

Metrics saved.


In [14]:
# The expanded ensemble is slightly worse! Let me investigate why.
# Check which N values changed between candidate_009 and this ensemble

print("Comparing candidate_009 (70.630478) vs new ensemble (70.630603):")
print("="*60)

changes = []
for n in range(1, 201):
    prev_trees, _ = load_trees_from_df(prev_best_df, n)
    prev_score = calculate_score(prev_trees)
    new_score = ensemble_scores[n]
    
    diff = new_score - prev_score
    if abs(diff) > 1e-9:
        changes.append((n, prev_score, new_score, diff, ensemble_sources[n]))

print(f"Total N values that changed: {len(changes)}")
for n, prev, new, diff, src in sorted(changes, key=lambda x: x[3]):
    direction = "BETTER" if diff < 0 else "WORSE"
    print(f"  N={n}: {prev:.6f} -> {new:.6f} ({diff:+.6f}) {direction} - from {src}")

Comparing candidate_009 (70.630478) vs new ensemble (70.630603):


Total N values that changed: 5
  N=165: 0.335569 -> 0.335569 (+0.000000) WORSE - from smartmanoj.csv
  N=42: 0.366839 -> 0.366839 (+0.000000) WORSE - from smartmanoj.csv
  N=166: 0.334819 -> 0.334820 (+0.000001) WORSE - from smartmanoj.csv
  N=134: 0.346472 -> 0.346475 (+0.000003) WORSE - from smartmanoj.csv
  N=138: 0.341028 -> 0.341149 (+0.000121) WORSE - from smartmanoj.csv


In [15]:
# Let me check the BEST possible score if we pick the absolute best for each N
# (ignoring source name conflicts)

print("Best possible score from all sources (ignoring overlaps):")
best_possible = 0
for n in range(1, 201):
    best_n = float('inf')
    for name in sources:
        if n in scores_by_source[name]:
            if scores_by_source[name][n] < best_n:
                best_n = scores_by_source[name][n]
    best_possible += best_n

print(f"Best possible (no overlap check): {best_possible:.6f}")
print(f"Current best (candidate_009): 70.630478")
print(f"Target: 68.919154")
print(f"Gap: {best_possible - 68.919154:.6f}")

# Now let's see which sources have the best scores for each N
print("\n\nBest source for each N (by raw score):")
for n in range(1, 201):
    best_n = float('inf')
    best_src = None
    for name in sources:
        if n in scores_by_source[name]:
            if scores_by_source[name][n] < best_n:
                best_n = scores_by_source[name][n]
                best_src = name
    
    # Compare to candidate_009
    prev_trees, _ = load_trees_from_df(prev_best_df, n)
    prev_score = calculate_score(prev_trees)
    
    if best_n < prev_score - 1e-9:
        print(f"  N={n}: {prev_score:.6f} -> {best_n:.6f} (improvement: {prev_score - best_n:.6f}) from {best_src}")

Best possible score from all sources (ignoring overlaps):
Best possible (no overlap check): 70.630478
Current best (candidate_009): 70.630478
Target: 68.919154
Gap: 1.711324


Best source for each N (by raw score):


  N=134: 0.346472 -> 0.346472 (improvement: 0.000000) from santa-2025.csv


  N=154: 0.334158 -> 0.334158 (improvement: 0.000000) from santa-2025.csv


  N=166: 0.334819 -> 0.334819 (improvement: 0.000000) from santa-2025.csv


In [ ]:
# Let's try running the C++ optimizer (sa_v1_parallel) on our current best
# to see if we can squeeze out any micro-improvements

import subprocess
import os

# Copy current best to working directory
import shutil
shutil.copy('/home/code/submission_candidates/candidate_009.csv', '/home/code/exploration/datasets/current_best.csv')

# Run the optimizer
optimizer_path = '/home/code/exploration/datasets/sa_v1_parallel'
input_path = '/home/code/exploration/datasets/current_best.csv'
output_path = '/home/code/exploration/datasets/optimized_output.csv'

# Make sure optimizer is executable
os.chmod(optimizer_path, 0o755)

# Run with high iterations
cmd = f'{optimizer_path} -i {input_path} -o {output_path} -n 30000 -r 5'
print(f'Running: {cmd}')
result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=600)
print(result.stdout)
print(result.stderr)